# Trabajando con pipelines

El `Pipeline` de Scikit-Learn es una herramienta que simplifica el proceso de construcción y evaluación de modelos de aprendizaje automático, al permitir que múltiples pasos del proceso se agrupen en una sola entidad. Aquí están algunas de sus ventajas:

1. **Simplicidad y Conveniencia:** 
   - Un `Pipeline` agrupa todos los pasos que se deben realizar antes de la etapa de predicción en una sola entidad. Esto simplifica el código y facilita su gestión, especialmente cuando hay múltiples pasos de preprocesamiento.

2. **Evitar Fugas de Datos:**
   - Al usar un `Pipeline`, las operaciones de preprocesamiento se aplican solo al conjunto de datos con el que se está trabajando en cada etapa (entrenamiento, validación o prueba). Esto ayuda a evitar fugas de datos, donde información del conjunto de prueba podría influir en el modelo durante el entrenamiento.

3. **Facilita la Validación Cruzada:**
   - El `Pipeline` permite evaluar todo el proceso de transformación y modelado mediante validación cruzada de una sola vez, asegurando que se utilice el mismo conjunto de datos en cada etapa del proceso.

4. **Optimización de Hiperparámetros:**
   - En combinación con funciones como `GridSearchCV` o `RandomizedSearchCV`, puedes optimizar los hiperparámetros de todo el `Pipeline`, no solo del estimador final. Esto es especialmente útil cuando los hiperparámetros de las etapas de preprocesamiento también son críticos para el rendimiento del modelo.

5. **Reproducibilidad y Portabilidad:** 
   - Un `Pipeline` agrupa todos los pasos en un objeto único. Esto hace que sea mucho más fácil compartir modelos entre diferentes miembros del equipo o incluso entre diferentes proyectos, ayudando a garantizar que todos los pasos se ejecuten de manera consistente.

6. **Facilita el Despliegue en Producción:** 
   - Una vez que se entrena un `Pipeline`, se puede guardar y cargar como un solo objeto, lo que simplifica mucho el proceso de mover un modelo desde un entorno de desarrollo a un entorno de producción.
   
Aquí hay un ejemplo simple de un Pipeline que incluye la imputación de valores faltantes, la normalización de características y la clasificación mediante un modelo de regresión logística:

In [3]:
y_train

array([2, 2, 1, 2, 0, 1, 1, 1, 2, 0, 1, 1, 2, 0, 1, 0, 0, 2, 2, 1, 1, 0,
       1, 0, 2, 1, 1, 2, 0, 0, 0, 2, 0, 0, 1, 2, 1, 0, 2, 1, 0, 2, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 2, 1, 1, 1, 0, 1, 1, 1, 2, 2, 0, 1, 2, 2, 1,
       1, 0, 1, 2, 2, 1, 2, 1, 1, 1, 0, 0, 2, 0, 2, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 2, 1, 1, 1, 2, 2, 1, 0, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 1,
       0, 1, 0, 2, 0, 0, 1, 0, 0, 2, 1, 0, 2, 2, 0, 0, 2, 2, 2, 1, 1, 1,
       1, 1, 1, 2, 0, 1, 1, 0, 1, 1])

In [1]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

data = load_wine()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

# Definir los pasos del pipeline
pipeline_steps = [
    ('imputer', SimpleImputer(strategy='mean')),   # Imputación de valores faltantes
    ('scaler', StandardScaler()),                 # Normalización de las características
    ('classifier', LogisticRegression())          # Modelo de clasificación
]

# Crear el pipeline
pipeline = Pipeline(pipeline_steps)

# Ajustar el pipeline a los datos de entrenamiento
pipeline.fit(X_train, y_train)

# Evaluar el pipeline en los datos de prueba
score = pipeline.score(X_test, y_test)

print("Score del Pipeline:", score)


Score del Pipeline: 1.0


In [2]:
# Especificar múltiples métricas de evaluación
scoring = ['precision_macro', 'recall_macro', 'f1_macro']

# Realizar la validación cruzada
scores = cross_validate(pipeline, X_train, y_train, cv=5, scoring=scoring)

# Imprimir los resultados
for metric_name, scores_array in scores.items():
    print(f'{metric_name}: {scores_array.mean():.4f}')

fit_time: 0.0172
score_time: 0.0105
test_precision_macro: 0.9801
test_recall_macro: 0.9810
test_f1_macro: 0.9795


La función cross_validate de Scikit-Learn es una herramienta que permite evaluar múltiples métricas de un estimador (como un modelo o un pipeline) mediante validación cruzada. A diferencia de cross_val_score, que solo devuelve los puntajes de evaluación, cross_validate permite especificar múltiples métricas y además devuelve información sobre el tiempo que tarda cada fase del proceso de evaluación.

Sus aspectos más relevantes son:

- Múltiples métricas
- Información de Tiempo
- Especifica número de folds

Respecto a lo "Macro", se debe una explicación:

Score de Macro-Average:

- Calcula la métrica para cada clase y luego toma el promedio sin ponderar las clases. Esto puede ser útil si te importa el rendimiento en **todas las clases por igual**, independientemente de su tamaño.

Score de Micro-Average:

- Calcula métricas globales sumando los verdaderos positivos, falsos positivos y falsos negativos. Es útil si te importa el **rendimiento global** sobre todas las clases.

### Un ejemplo con varios modelos (foco en modelamiento)

In [4]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

data = load_wine()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2)

In [5]:
#Los pipeline se crean de la siguiente manera:

# Regresión Logística
pipe_logistic = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression())
])

# Random Forest
pipe_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier())
])

# AdaBoost
pipe_adaboost = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', AdaBoostClassifier())
])


In [7]:
# Se definen los espacios de hiperparámetros para cada modelo:


# Hiperparámetros para la Regresión Logística
param_grid_logistic = {
    'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'classifier__max_iter': [500,1000,1500]
}

# Hiperparámetros para el Random Forest
param_rf = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10]
}


param_grid_ada = {
    'classifier__n_estimators': [50, 100, 150],
    'classifier__learning_rate': [0.01, 0.1, 1]
}


In [8]:
#HPO

# Regresión Logística
search_logistic = RandomizedSearchCV(pipe_logistic, param_grid_logistic, n_iter=10, cv=5)
search_logistic.fit(X_train, y_train)

# Random Forest
search_rf = RandomizedSearchCV(pipe_rf, param_rf, n_iter=10, cv=5, random_state=42)
search_rf.fit(X_train, y_train)

# AdaBoost
search_adaboost = RandomizedSearchCV(pipe_adaboost, param_grid_ada, n_iter=10, cv=5)
search_adaboost.fit(X_train, y_train)


C:\Users\aleph\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                             ('classifier',
                                              AdaBoostClassifier())]),
                   param_distributions={'classifier__learning_rate': [0.01, 0.1,
                                                                      1],
                                        'classifier__n_estimators': [50, 100,
                                                                     150]})

In [9]:
# Imprimir los resultados
print("Mejores parámetros Regresión Logística: ", search_logistic.best_params_)
print("Precisión Regresión Logística: ", search_logistic.score(X_test, y_test))

print("Mejores parámetros Random Forest: ", search_rf.best_params_)
print("Precisión Random Forest: ", search_rf.score(X_test, y_test))

print("Mejores parámetros AdaBoost: ", search_adaboost.best_params_)
print("Precisión AdaBoost: ", search_adaboost.score(X_test, y_test))


Mejores parámetros Regresión Logística:  {'classifier__max_iter': 500, 'classifier__C': 10}
Precisión Regresión Logística:  0.9722222222222222
Mejores parámetros Random Forest:  {'classifier__n_estimators': 50, 'classifier__min_samples_split': 5, 'classifier__max_depth': 30}
Precisión Random Forest:  1.0
Mejores parámetros AdaBoost:  {'classifier__n_estimators': 150, 'classifier__learning_rate': 0.01}
Precisión AdaBoost:  1.0


In [10]:
#Calcular y Comparar Métricas de Evaluación
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Predecir las etiquetas del conjunto de prueba
y_pred_logistic = search_logistic.predict(X_test)
y_pred_rf = search_rf.predict(X_test)
y_pred_ada = search_adaboost.predict(X_test)

# Comparar la precisión
print("Precisión de Regresión Logística:", accuracy_score(y_test, y_pred_logistic))
print("Precisión de Random Forest:", accuracy_score(y_test, y_pred_rf))
print("Precisión de AdaBoost:", accuracy_score(y_test, y_pred_ada))

# Comparar F1-score
print("F1-score de Regresión Logística:", f1_score(y_test, y_pred_logistic, average='weighted'))
print("F1-score de Random Forest:", f1_score(y_test, y_pred_rf, average='weighted'))
print("F1-score de AdaBoost:", f1_score(y_test, y_pred_ada, average='weighted'))


Precisión de Regresión Logística: 0.9722222222222222
Precisión de Random Forest: 1.0
Precisión de AdaBoost: 1.0
F1-score de Regresión Logística: 0.9729344729344729
F1-score de Random Forest: 1.0
F1-score de AdaBoost: 1.0


In [ ]:
#### ahoa con adaboost